In [1]:
### Baseline recommender - napove izdelke ki jih je kupec kupil v redzadnjem nakupu ###

In [2]:
#importi
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

In [3]:
parse_dates = ['datum']

print("loading nakupi.csv...")
orders = pd.read_csv("Podatki/nakupi.csv",dtype={ #nrows=500000
        'kupec': np.uint32,
        'st_racuna': np.uint32,
        'artikel': np.uint32,
        'letak': np.bool},
        parse_dates=parse_dates)
print("done")

loading nakupi.csv...
done


In [4]:
#Preparing data sets train/test
print("Preparing data")

tmp = pd.DataFrame()
orders['last_order'] = orders.groupby('kupec')['st_racuna'].transform(max)
tmp = orders.assign(last=orders.st_racuna == orders.last_order)
tmp = tmp.assign(second_last=orders.st_racuna == orders.last_order-1)

### TEST ###
test_second_last = tmp.loc[tmp['second_last'] == True] #  ostanejo samo predzadnji zadnji nakup ali ne
test_second_last = test_second_last.groupby('kupec').agg({'artikel':lambda x: list(x)}) # transforms real artikels to list

test_last = tmp.loc[tmp['last'] == True] # ostanejo samo zadni nakupi
test_for_f1 = test_last.groupby('kupec').agg({'artikel':lambda x: list(x)}) # transforms real artikels to list
print("done")

Preparing data
done


In [5]:
test_second_last.head(5)

,artikel
kupec,
0,"[440, 1503, 1652, 1328, 1586, 5826, 271, 2364,..."
1,"[247, 5438, 195, 4703, 4434, 3221, 5669, 115]"
2,"[34, 4512, 968, 1632, 5216, 1945, 2513, 497, 1..."
3,"[3026, 3022, 1582, 12549, 60, 2419]"
4,"[1351, 604, 4282, 53]"


In [6]:
test_for_f1.head(5)

,artikel
kupec,
0,"[4294, 1119, 2203, 1506, 6406, 1226, 2575, 158..."
1,[4691]
2,"[877, 3688, 9521, 9005, 124, 4512, 41, 10780, ..."
3,"[5428, 1582, 6274, 397, 15612, 2245, 3022, 55,..."
4,"[227, 1351, 9191, 322, 4201]"


In [7]:
### TESTING ####

def eval_fun(labels, preds):
    labels = labels
    preds = preds#.split(' ')
    rr = (np.intersect1d(labels, preds))
    precision = np.float(len(rr)) / len(preds)
    recall = np.float(len(rr)) / len(labels)
    try:
        f1 = 2 * precision * recall / (precision + recall)
        #print("   p={0:.2f} r={0:.2f} f={0:.2f} preds: ".format(precision, recall, f1, preds) + str(rr))
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)


###########
test_for_f1 = pd.merge(test_for_f1, test_second_last, how='left', on='kupec')
print("done")
len(test_for_f1)

done


19650

In [8]:
test_for_f1.head(5) #artikel_x=labels, artikel_y=preds

,artikel_x,artikel_y
kupec,,
0,"[4294, 1119, 2203, 1506, 6406, 1226, 2575, 158...","[440, 1503, 1652, 1328, 1586, 5826, 271, 2364,..."
1,[4691],"[247, 5438, 195, 4703, 4434, 3221, 5669, 115]"
2,"[877, 3688, 9521, 9005, 124, 4512, 41, 10780, ...","[34, 4512, 968, 1632, 5216, 1945, 2513, 497, 1..."
3,"[5428, 1582, 6274, 397, 15612, 2245, 3022, 55,...","[3026, 3022, 1582, 12549, 60, 2419]"
4,"[227, 1351, 9191, 322, 4201]","[1351, 604, 4282, 53]"


In [9]:
res = list()
for entry in test_for_f1.itertuples():
    #print("order:{} preds:{}".format(entry[1], entry[2]))
    res.append(eval_fun(entry[1], entry[2]))

res = pd.DataFrame(np.array(res), columns=['precision', 'recall', 'f1'])

In [10]:
res.describe()

,precision,recall,f1
count,19650.000000,19650.000000,19650.000000
mean,0.096010,0.096174,0.084006
std,0.174843,0.175351,0.143900
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.142857,0.142857,0.142857
max,1.000000,1.000000,1.000000
